# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,afaahiti,-17.7500,-149.2833,25.93,78,68,4.65,PF,1691348134
1,1,saipan,15.1355,145.7010,27.39,100,0,0.45,MP,1691348119
2,2,waitangi,-43.9535,-176.5597,8.94,70,100,5.65,NZ,1691348114
3,3,whitehorse,60.7161,-135.0538,22.43,46,20,5.66,CA,1691347892
4,4,longyearbyen,78.2186,15.6401,10.91,76,75,3.60,SJ,1691348141


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,123,dalandzadgad,43.5708,104.4250,25.22,17,0,2.62,MN,1691348301
172,172,ongjin,37.9347,125.3619,23.84,93,0,2.07,KP,1691348307
195,195,tuapse,44.1125,39.0742,26.18,71,0,1.14,RU,1691348311
252,252,sinop,41.7711,34.8709,22.98,58,0,1.44,TR,1691348319
259,259,amasra,41.7463,32.3863,26.50,51,0,3.49,TR,1691348319
266,266,constantia,44.1833,28.6500,23.38,84,0,4.45,RO,1691348116
370,370,ajaccio,42.0000,8.9167,23.03,53,0,2.57,FR,1691348093
372,372,sines,37.9562,-8.8698,24.60,62,0,1.80,PT,1691348332
376,376,dom pedrito,-30.9828,-54.6731,25.66,62,0,4.37,BR,1691348162
391,391,hailey,43.5196,-114.3153,26.60,46,0,2.24,US,1691348335


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
123,dalandzadgad,MN,43.5708,104.4250,17,
172,ongjin,KP,37.9347,125.3619,93,
195,tuapse,RU,44.1125,39.0742,71,
252,sinop,TR,41.7711,34.8709,58,
259,amasra,TR,41.7463,32.3863,51,
266,constantia,RO,44.1833,28.6500,84,
370,ajaccio,FR,42.0000,8.9167,53,
372,sines,PT,37.9562,-8.8698,62,
376,dom pedrito,BR,-30.9828,-54.6731,62,
391,hailey,US,43.5196,-114.3153,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dalandzadgad - nearest hotel: Хан Уул буудал
ongjin - nearest hotel: No hotel found
tuapse - nearest hotel: Отель Роза Ветров
sinop - nearest hotel: No hotel found
amasra - nearest hotel: Liman Hotel
constantia - nearest hotel: Hotel Chérica
ajaccio - nearest hotel: Hôtel Sole e Monte
sines - nearest hotel: Sinerama
dom pedrito - nearest hotel: Hotel Alexandre
hailey - nearest hotel: Fairfield Inn
novyy turtkul' - nearest hotel: To'rtku'l miymanxanasi
ilhabela - nearest hotel: Vila Kebaya
targovishte - nearest hotel: Тера Европа
gijang - nearest hotel: 힐튼호텔 부산
provadia - nearest hotel: No hotel found
revelstoke - nearest hotel: Grizz Hotel
montanaro - nearest hotel: Fortyfive
mirto - nearest hotel: Hotel La Meridiana
kamennogorsk - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
123,dalandzadgad,MN,43.5708,104.4250,17,Хан Уул буудал
172,ongjin,KP,37.9347,125.3619,93,No hotel found
195,tuapse,RU,44.1125,39.0742,71,Отель Роза Ветров
252,sinop,TR,41.7711,34.8709,58,No hotel found
259,amasra,TR,41.7463,32.3863,51,Liman Hotel
266,constantia,RO,44.1833,28.6500,84,Hotel Chérica
370,ajaccio,FR,42.0000,8.9167,53,Hôtel Sole e Monte
372,sines,PT,37.9562,-8.8698,62,Sinerama
376,dom pedrito,BR,-30.9828,-54.6731,62,Hotel Alexandre
391,hailey,US,43.5196,-114.3153,46,Fairfield Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)